In [1]:
import shutil
print(shutil.which("ffmpeg"))


C:\Users\impos\.conda\envs\bassem_env\Library\bin\ffmpeg.EXE


In [2]:
import os
import subprocess

# Check if eSpeak works inside Python
result = subprocess.run(["espeak", "--version"], capture_output=True, text=True)
print(result.stdout)  # Should display eSpeak version


speak text-to-speech: 1.48.03  04.Mar.14   Data at: C:\eSpeak\espeak-data



In [3]:
import os
os.environ["ESPEAK_LIBRARY"] = "C:/espeak/command_line/espeak.exe"
os.environ["PATH"] += os.pathsep + "C:/espeak/command_line/"


In [ ]:
from phonemizer import phonemize

sentence = "Hey dear, how was your day"
output = phonemize(sentence, language="en-us", backend="espeak")
print(output)


In [5]:
import os
import json
import librosa
import numpy as np
from phonemizer import phonemize
from pydub.utils import mediainfo
import subprocess

# Ensure eSpeak is recognized inside Python
os.environ["PATH"] += os.pathsep + "C:/espeak/command_line/"
os.environ["PHONEMIZER_ESPEAK_LIBRARY"] = "C:/espeak/command_line/espeak.exe"

# Function to get audio duration
def get_audio_duration(audio_path):
    audio_info = mediainfo(audio_path)
    duration = float(audio_info['duration'])
    return duration

# Function to get phonemes for a sentence
# Function to get phonemes for a sentence
def get_phonemes(sentence):
    phoneme_output = phonemize(sentence, language="en-us", backend="espeak")  # Remove espeak_path
    phonemes = phoneme_output.split()
    return phonemes


# Function to generate timestamps for phonemes
def generate_mouthCues(sentence, audio_path):
    duration = get_audio_duration(audio_path)
    phonemes = get_phonemes(sentence)

    # Load audio and get time intervals
    y, sr = librosa.load(audio_path, sr=None)
    num_frames = len(phonemes)
    frame_duration = duration / num_frames

    mouthCues = []
    for i, phoneme in enumerate(phonemes):
        start_time = round(i * frame_duration, 2)
        end_time = round((i + 1) * frame_duration, 2)
        mouthCues.append({"start": start_time, "end": end_time, "value": phoneme})

    return mouthCues

# Function to generate JSON file
def generate_lipsync_json(sentence, audio_path, save_path):
    mouthCues = generate_mouthCues(sentence, audio_path)
    duration = get_audio_duration(audio_path)

    data = {
        "metadata": {"duration": round(duration, 2)},
        "mouthCues": mouthCues
    }

    with open(save_path, 'w') as file:
        json.dump(data, file, indent=4)

    print(f"Lipsync data saved to {save_path}")

# 🔹 Usage Example
sentence = "Hey dear, how was your day"
audio_path = "C:/new grad project (farid version)/grad/backend/audios/intro_0.wav"
save_path = "C:/new grad project (farid version)/grad/backend/audios/lipsync_intro_1.json"

generate_lipsync_json(sentence, audio_path, save_path)


RuntimeError: espeak not installed on your system